In [1]:
from imgseries import ImgSeries
from imgseries import CONFIG
from imgseries import TransformParameter

# Change order and/or number of transforms

One can see the current ordering of transforms with:

In [2]:
CONFIG['transform order']

('grayscale', 'rotation', 'crop', 'filter', 'subtraction', 'threshold')

One can use them in a different order and also reduce the number of possible transforms by using the `transforms` argument in ImgSeries

In [3]:
images = ImgSeries('../data/img1', transforms=('rotation', 'filter', 'crop'))
images.transforms


OrderedDict([('rotation', Rotation {}),
             ('filter', Filter {}),
             ('crop', Crop {})])

In this case, the other transforms are not available, e.g.

In [4]:
try:
    images.threshold
except AttributeError:
    print("Threshold transform does not exist")

Threshold transform does not exist


# Create a user-defined transform

It is also possible to add custom transforms.
Here, we will show how to proceed, with the example of a new transform that consists in multiplying the pixel values with a constant.

## Write custom transform class

### Mandatory

The class must inherit from `TransformParameter` and define the following attributes:

- `name` (class attribute): name of the transform; will be used as the name of an attribute, and thus needs to follow usual rules (lowercase, no starting with a number, etc.)

- `apply()`: method which takes an image array as an argument and returns another (processed) image array.

- The important values governing the transform (e.g. angle for rotation, etc.) must be stored in the already-defined (but empty) dict `self.data`; This will allow automatic saving/loading of transform parameters from JSON file.

### Optional

- For convenience, it can be useful to wrap the transform parameters (contained in `self.data` into settable properties, see e.g. `coefficient` below)

- If the transform parameters can be chosen interactively / graphically, the user should provide a `define()` method with a `num` argument, which corresponds to the image number on which to do the interactive process.

*Note*: The `TransformParameter` class also automatically defines the attribute `self.img_series`, which allows the user to use all `ImgSeries` attributes, methods etc.

## Add/remove transform to available transforms

This is done very simply by adding the user-defined class with
```python
ImgSeries.add_transform(TransformClass)
```

A custom, default order can be supplied:
```python
ImgSeries.add_transform(TransformClass, order=3)
```

Note that the new transform is added to the configuration of the `imgseries` package, and is thus available for all instances of `ImgSeries` and `ImgStack`, as long as the python session is not closed.

In order to remove the transform:
```python
ImgSeries.remove_transform(TransformClass)
```

## Examples

### Minimal example

In [5]:
class Multiply(TransformParameter):

    name = 'multiply'

    def apply(self, img):
        new_img = self.data['coefficient'] * img
        return new_img.astype(img.dtype)  # to preserve type of image e.g. uint8

In [6]:
ImgSeries.add_transform(Multiply, order=1)
CONFIG['transform order']

('grayscale',
 'multiply',
 'rotation',
 'crop',
 'filter',
 'subtraction',
 'threshold')

In [7]:
images = ImgSeries('../data/img1')
images.transforms


OrderedDict([('grayscale', Grayscale {}),
             ('multiply', Multiply {}),
             ('rotation', Rotation {}),
             ('crop', Crop {}),
             ('filter', Filter {}),
             ('subtraction', Subtraction {}),
             ('threshold', Threshold {})])

In [8]:
images.multiply.data['coefficient'] = 2
img0 = images.read(num=3, transform=False)
img1 = images.read(num=3)

print(images.active_transforms)
print(img0.mean(), img1.mean())

['multiply']
77.0014144736842 151.01412081339714


In [9]:
ImgSeries.remove_transform(Multiply)
CONFIG['transform order']

('grayscale', 'rotation', 'crop', 'filter', 'subtraction', 'threshold')

### With settable properties

In [10]:
class Multiply(TransformParameter):

    name = 'multiply'

    def apply(self, img):
        new_img = self.coefficient * img
        return new_img.astype(img.dtype)  # to preserve type of image e.g. uint8

    @property
    def coefficient(self):
        return self.data['coefficient']

    @coefficient.setter
    def coefficient(self, value):
        self.data['coefficient'] = value

In [11]:
ImgSeries.add_transform(Multiply, order=3)
images = ImgSeries('../data/img1')

In [12]:
images.multiply.coefficient = 2
img0 = images.read(num=3, transform=False)
img1 = images.read(num=3)

print(images.active_transforms)
print(img0.mean(), img1.mean())

['multiply']
77.0014144736842 151.01412081339714


# Apply same transform two (or more) times

By default, transforms are applied only once.
If there is a need to apply the same type of transform twice, then the easiest way is to define a new class that simply inherits from the transform class, and to name it differently.

For example, to achieve two rotations:

In [13]:
from imgseries import Rotation

class Rotation2(Rotation):
    name = 'rotation2'

ImgSeries.add_transform(Rotation2)
CONFIG['transform order']

('grayscale',
 'rotation',
 'crop',
 'multiply',
 'filter',
 'subtraction',
 'threshold',
 'rotation2')

In [14]:
ImgSeries.remove_transform(Rotation2)
CONFIG['transform order']

('grayscale',
 'rotation',
 'crop',
 'multiply',
 'filter',
 'subtraction',
 'threshold')